In [1]:
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langchain_google_genai import ChatGoogleGenerativeAI
from tqdm import tqdm
from typing import TypedDict, Annotated
from dotenv import load_dotenv
from langgraph.checkpoint.memory import InMemorySaver
from langchain_core.messages import HumanMessage, BaseMessage

In [2]:
load_dotenv()

True

In [3]:
llm = ChatGoogleGenerativeAI(
    model = 'gemini-2.5-flash'
)

In [4]:
class ChatState(TypedDict):
    messages: Annotated[list[BaseMessage], add_messages] 

In [ ]:
def chatNode(state: ChatState):
    messages = state['messages']
    
    response = llm.invoke(messages)

    return {'messages' : [response]}

In [15]:
checkpointer = InMemorySaver()

graph = StateGraph(ChatState)

graph.add_node('chat_node', chatNode)

graph.add_edge(START, 'chat_node')
graph.add_edge('chat_node', END)

chat = graph.compile(checkpointer=checkpointer)


In [16]:
thread_id = '1'

while True:

    user_message = input("Type Here : ")

    print('User : ', user_message)

    config = {
        'configurable': {
            'thread_id': thread_id
        }
    }

    if user_message.strip().lower() in ['exit', 'quit', 'bye']:
        break 

    response = chat.invoke({'messages': [HumanMessage(content=user_message)]}, config=config)
    
    print('AI : ', response['messages'][-1].content)

User :  Hii I am Raj
[HumanMessage(content='Hii I am Raj', additional_kwargs={}, response_metadata={}, id='d5101d74-d1f6-49a8-9d33-c18eb4fb60b5')]
[HumanMessage(content='Hii I am Raj', additional_kwargs={}, response_metadata={}, id='d5101d74-d1f6-49a8-9d33-c18eb4fb60b5')]
AI :  Hi Raj! Nice to meet you.

I'm an AI assistant, ready to help you with whatever you need. What can I do for you today?
User :  what is my name
[HumanMessage(content='Hii I am Raj', additional_kwargs={}, response_metadata={}, id='d5101d74-d1f6-49a8-9d33-c18eb4fb60b5'), AIMessage(content="Hi Raj! Nice to meet you.\n\nI'm an AI assistant, ready to help you with whatever you need. What can I do for you today?", additional_kwargs={}, response_metadata={'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': [], 'model_provider': 'google_genai'}, id='lc_run--019b9e9c-fd49-70b1-8ae6-526b7711344e-0', usage_metadata={'input_tokens': 6, 'output_tokens': 322, 'total_tokens': 328, 'input_token_details':